# <font color= 'Green'>Optimal System Identification for LIGO</font>
### of linear, time-invariant (LTI) systems
***
* the LIGO Control Systems Working Group wiki: https://wiki.ligo.org/CSWG/OptTF
* Rana's public GitHub page on LIGO Controls problems: https://github.com/rxa254/LIGO-Controls-Problems
***
This notebook is meant to give an introduction to a couple of kinds of sysID problems in LIGO. The goal is to generate a few specific strategies to do this better for important cases, and eventually to make a more general tool for this.

# Overview
## The Identification Problem
We would like to know what our physical plants (optics, suspensions, electronics, cavities) are doing. In nearly all cases, we need not consider the nonlinearity or time-dependence of the plant (notable exceptions due to thermal processes and slow drift and changes in laser power).

We approach this problem by making Transfer Functions (TF) of the system that we are interested in.

How to make a TF measurement:
1. with enough SNR at the frequencies of interest
1. without saturating the actuators too much
1. within a small enough amount of time (so that our precious commissioning / observation time is not squandered)

In [5]:
%matplotlib inline
# Import packages.
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np




In [6]:
# Let's define the system to be 'identified'

# pendulum
zz = []
f_p = 1
a_p = 60
pp = [f_p*np.exp(1j*a_p*np.pi/180)]
pp = [pp[0],np.conj(pp[0])]
pend = signal.ZerosPolesGain(zz,pp,1)

In [ ]:
# here we calculate the waveform

But a good way to represent the audio files is through a spectrogram (short time fourier transform), google this if you dont know what this means. So we plot the spectrograms of our separation.

In [ ]:
plt.figure()
f, t, S = signal.spectrogram(s[0][0])
plt.pcolormesh(t, f, S)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram of Output 1')

plt.figure()
f, t, S = signal.spectrogram(s[1][0])
plt.pcolormesh(t, f, S)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram of Output 2')

# Storing numpy array as audio
wavfile.write('fastICA/out1.wav', samplingRate, np.transpose(s[0][0]))
wavfile.write('fastICA/out2.wav', samplingRate, np.transpose(s[1][0]))

In [ ]:
from IPython.display import Audio
print('Mixed Signal 1')
Audio("fastICA/mic1.wav")

In [ ]:
print('Mixed Signal 2')
Audio("fastICA/mic2.wav")

In [ ]:
print('Original separated signal 1')
Audio("fastICA/source1.wav")

In [ ]:
print('Original separated signal 2')
Audio("fastICA/source1.wav")

In [ ]:
print('Separated signal 1 (output)')
Audio("fastICA/out1.wav")

In [ ]:
print('Separated signal 2 (output)')
Audio("fastICA/out2.wav")

## <font color='blue'>FOBI</font>

FOBI stands for Fourth Order Blind Identification. 

As the name suggests, it is based on considering fourth order moments and using them as a metric for Gaussianity. 
Covariance, variance and the covariance matrix are second order moments. 
Fourth order moments are usually referred to by the term kurtosis. There are other ICA methods that explicitly work with kurtosis, but not so with FOBI. 

FOBI is a very elegant method that works without the need to search a solution space for the indepenedent components(as is the case with fastICA and other negentropy or maximum-likelihood based ICA approaches). Rather, it provides an explicit algebraic formula for the independent components. 

It has some striking and appealing similarities to PCA. FOBI is based on the following amazing fact:

**The weights of the independent components are the eigenvectors of the matrix $cov(|X'|X')$. **

The statement requires some explanation. 
$X'$ is the matrix obtained after preprocessing $X$ (mean-centered and whitened).

$|X'|$ needs to be explained. Remember that each column of $X'$ corresponds to the (whitened) set of amplitudes recorded at some instant of time by the **m** microphones. If we take this column to be a vector, then the norm (aka the modulus) of this vector is well defined. We find the norm of each column of **X'** and put these calculated norms in a row matrix. This is the matrix $|X'|$. 
Hence, $|X'|$ is simply the norm of the matrix $X$ taken along the "column-axis".

But here comes a caveat: The matrix $|X'|X'$ is not formed by the normal matrix multiplication between $|X'|$ and $X'$. We have written it this way only for notational convnenience. Instead, we are using the elements of $|X'|$ as weights for the columns of $X'$. That is, the matrix $|X'|X'$ is formed as follows:

The **i**th column of $|X'|X'$ is formed by a scalar multiplication of the **i**th element of the **1 x m** row matrix $|X'|$ with the **i**th column of the **m x n** matrix $X'$ (each element within this column is thus multiplied by the same scalar value equal to the **i**th element of $|X'$).   

Hence, $|X'|X'$ is a weighted version of $X'$. It is easy to think of each column of $X'$ as a vector in an m-dimensional space. Then $|X'|$ contains the norms of the **m** columns of $X'$ i.e vectors. Multiplying each vector (column) by it's norm and storing these column vectors as columns in a new matrix yields $|X'|X'$.

And what does the statement itself mean? Well, remember how we said that $X = AS$. After whitening, we can say that $X' = A'S'$.
Here, $S'$ corresponds to the source signals itself, but their amplitude has been normalised. (Recall how ICA can only recover the signals upto to a multiplicative factor.)  

The columns of $A'$ are the weights being referred to in the statement above. They are called weights because the each row of $S'$ is a source signal. Upon performing matrix multiplication between $A'$ and $S'$, we can see that $X' = \sum_{1}^{m}A'_{i}S'_{i}$. Note that $A'_{i}$ are column vectors and $S'_{i}$ are row vectors. Hence, the columns of $A'$ act as weights for the source signals. 

Hence, FOBI allows us to directly compute the matrix $A'$ which as expected, turns out to be orthogonal. (The result says that the columns of $A'$ are orthogonal eigenvectors of the matrix $cov(|X'|X')$. Hence, $A'$ is an orthogonal matrix.) Finding the signals is now trivial. $S' = A'^{-1}X' = A'^{T}X'$. (The inverse of an orthogonal matrix is simply it's transpose.)

Recall that in PCA, the principal components were eigenvectors of the matrix $cov(X)$ and they were all orthogonal. Here, the columns of $A'$, which are the weights for the source signals are the eigenvectors of the matrix  $cov(|X'|X')$. And the weights are orthogonal to one another! 

The proof of this result and a good explanation of the FOBI algorithm can be found in the original paper by the inventor of the FOBI algorithm Jean Cardoso, [here](http://perso.telecom-paristech.fr/~Cardoso/Papers.PDF/icassp89.pdf). The notation used in the paper is rather different from the one that we have been using so far and can be rather confusing. The proof itself is however extremely elegant. 

Also, refer to the paper to better understand why it involves fourth order moments. Basically, if $X'$ were a simple vector, then $cov(|X'|X')$ is fourth order in $X'$. This extends to $X'$ being a matrix.

So the FOBI algorithm in summary is:  

1) Obtain the data matrix *X*.

2) Subtract off the mean to center *X*.

3) Whiten the matrix *X* to obtain the matrix *X'*.

4) Compute the matrix  $cov(|X'|X')$. 

5) Find the eigenvectors of  $cov(|X'|X')$.

6) Store the eigenvectors as columns of a matrix $Y$. ($Y$ is just $A'$)

7) Retrieve the original signals as $S' = Y^{T}X'$. 


The code for the solvong the Cocktail Party Problem based on the FOBI algorithm is given below. In the code, we have used slightly different notation in some places. For instance, x for X and xn for X'. 

In [ ]:
%matplotlib inline
"""
Cocktail Party Problem solved via Independent Component Analysis.
The Fourth Order Blind Identification(FOBI) ICA is implemented here.
"""
# Import packages.
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from scipy.io import wavfile
from scipy import linalg as LA

# Input the data from the first receiver.
samplingRate, signal1 = wavfile.read('FOBI/Sounds/mix1.wav')
print "Sampling rate = ", samplingRate
print "Data type is ", signal1.dtype

# Convert the signal so that amplitude lies between 0 and 1.
signal1 = signal1 / 255.0 - 0.5  # uint8 takes values from 0 to 255

# Output information about the sound samples.
a = signal1.shape
n = a[0]
print "Number of samples: ", n
n = n * 1.0

# Input data from the second receiver and standardise it's amplitude.
samplingRate, signal2 = wavfile.read('FOBI/Sounds/mix2.wav')
signal2 = signal2 / 255.0 - 0.5  # uint8 takes values from 0 to 255

# x is our initial data matrix.
x = [signal1, signal2]

# Plot the signals from both sources to show correlations in the data.
plt.figure()
plt.plot(x[0], x[1], '*b')
plt.ylabel('Signal 2')
plt.xlabel('Signal 1')
plt.title("Original data")

# Calculate the covariance matrix of the initial data.
cov = np.cov(x)
# Calculate eigenvalues and eigenvectors of the covariance matrix.
d, E = LA.eigh(cov)
# Generate a diagonal matrix with the eigenvalues as diagonal elements.
D = np.diag(d)

Di = LA.sqrtm(LA.inv(D))
# Perform whitening. xn is the whitened matrix.
xn = np.dot(Di, np.dot(np.transpose(E), x))

# Plot whitened data to show new structure of the data.
plt.figure()
plt.plot(xn[0], xn[1], '*b')
plt.ylabel('Signal 2')
plt.xlabel('Signal 1')
plt.title("Whitened data")


In [ ]:
# Perform FOBI.
norm_xn = LA.norm(xn, axis=0)
norm = [norm_xn, norm_xn]

cov2 = np.cov(np.multiply(norm, xn))

d_n, Y = LA.eigh(cov2)

source = np.dot(np.transpose(Y), xn)

# Plot the separated sources.
time = np.arange(0, n, 1)
time = time / samplingRate
time = time * 1000  # convert to milliseconds

plt.figure()
plt.subplot(2, 2, 1).set_axis_off()
plt.plot(time, source[0], color='k')
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.title("Generated signal 1")

plt.subplot(2, 2, 2).set_axis_off()
plt.plot(time, source[1], color='k')
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.title("Generated signal 2")

# Plot the actual sources for comparison.
samplingRate, orig1 = wavfile.read('FOBI/Sounds/source1.wav')
orig1 = orig1 / 255.0 - 0.5  # uint8 takes values from 0 to 255

plt.subplot(2, 2, 3).set_axis_off()
plt.plot(time, orig1, color='k')
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.title("Original signal 1")

samplingRate, orig2 = wavfile.read('FOBI/Sounds/source2.wav')
orig2 = orig2 / 255.0 - 0.5  # uint8 takes values from 0 to 255

plt.subplot(2, 2, 4).set_axis_off()
plt.plot(time, orig2, color='k')
plt.ylabel('Amplitude')
plt.xlabel('Time (ms)')
plt.title("Original signal 2")

Again we plot the spectrograms

In [ ]:
plt.figure()
f, t, S = signal.spectrogram(source[0])
plt.pcolormesh(t, f, S)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram of Output 1')

plt.figure()
f, t, S = signal.spectrogram(source[1])
plt.pcolormesh(t, f, S)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram of Output 2')

# Storing numpy array as audio
wavfile.write('FOBI/Sounds/out1.wav', samplingRate, np.transpose(source[0]))
wavfile.write('FOBI/Sounds/out2.wav', samplingRate, np.transpose(source[1]))

In [ ]:
from IPython.display import Audio
print('Mixed Signal 1')
Audio("FOBI/Sounds/mix1.wav")

In [ ]:
print('Mixed Signal 2')
Audio("FOBI/Sounds/mix2.wav")

In [ ]:
print('Original separated sound 1')
Audio("FOBI/Sounds/source1.wav")

In [ ]:
print('Original separated sound 2')
Audio("FOBI/Sounds/source2.wav")

In [ ]:
print('Separated signal 1 (output)')
Audio("FOBI/Sounds/out1.wav")

In [ ]:
print('Separated signal 1 (output)')
Audio("FOBI/Sounds/out2.wav")

## Conclusion

* Need help in writing the code to do this

## References

The Sys ID book by Pintelon and Shoukens:
https://books.google.com/books?id=3lGJWtjGDzsC

SysID classroom exercises:
https://caltech.tind.io/record/748967?ln=en

How to take the frequency response measurement and find the plant parameters:

["Parameter Estimation and Model Order Identification of LTI Systems"](https://lmgtfy.com/?q=10.0.3.248%2Fj.ifacol.2016.07.333)

How to estimate the covariance matrix:

How to iterate the multi-sine excitation waveform based on the matrix: